In [1]:
import org.apache.spark.sql.types.{StructType, StructField, StringType}
import org.apache.spark.sql.functions.{col}
import org.apache.spark.sql._
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://10.0.0.232:4041
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1679770721642)
SparkSession available as 'spark'


import org.apache.spark.sql.types.{StructType, StructField, StringType}
import org.apache.spark.sql.functions.col
import org.apache.spark.sql._
import spark.implicits._


In [5]:
val graph = Seq(("1","2"),("1","3"),("2","3"),("2","1"))
val graphRDD = sc.parallelize(graph)

graph: Seq[(String, String)] = List((1,2), (1,3), (2,3), (2,1))
graphRDD: org.apache.spark.rdd.RDD[(String, String)] = ParallelCollectionRDD[1] at parallelize at <console>:34


In [29]:
def seqOp = (accumulator: List[String], element: String) => 
    accumulator:+element

def combOp = (accumulator1: List[String], accumulator2: List[String]) => 
    accumulator1 ++ accumulator2

val zeroVal = ()
val following = graphRDD.aggregateByKey(List[String]())(seqOp, combOp)

seqOp: (List[String], String) => List[String]
combOp: (List[String], List[String]) => List[String]
zeroVal: Unit = ()
following: org.apache.spark.rdd.RDD[(String, List[String])] = ShuffledRDD[16] at aggregateByKey at <console>:41


In [30]:
following.collect()

res6: Array[(String, List[String])] = Array((1,List(2, 3)), (2,List(3, 1)))


In [31]:
def seqOp = (accumulator: List[String], element: String) => 
    accumulator:+element

def combOp = (accumulator1: List[String], accumulator2: List[String]) => 
    accumulator1 ++ accumulator2

val zeroVal = ()
val followers = graphRDD.map{case(k,v) => (v,k)}.aggregateByKey(List[String]())(seqOp, combOp)

seqOp: (List[String], String) => List[String]
combOp: (List[String], List[String]) => List[String]
zeroVal: Unit = ()
followers: org.apache.spark.rdd.RDD[(String, List[String])] = ShuffledRDD[18] at aggregateByKey at <console>:41


In [32]:
followers.collect()

res7: Array[(String, List[String])] = Array((1,List(2)), (2,List(1)), (3,List(1, 2)))


In [76]:
val users = graphRDD.flatMap{ case (k,v) => List(k,v) }.distinct
val user_count = sc.broadcast(users.count())
users.collect()

users: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[62] at distinct at <console>:34
user_count: org.apache.spark.broadcast.Broadcast[Long] = Broadcast(47)
res28: Array[String] = Array(1, 2, 3)


In [77]:
user_count.value

res29: Long = 3


In [85]:
val ranks = users.toDF().withColumn("initial_rank", lit(1.0/user_count.value)).rdd

ranks: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[80] at rdd at <console>:33


In [86]:
ranks.collect()

res34: Array[org.apache.spark.sql.Row] = Array([1,0.3333333333333333], [2,0.3333333333333333], [3,0.3333333333333333])
